In [1]:
import gym
from collections import namedtuple
import numpy as np
from torch.utils.tensorboard import SummaryWriter

import torch
import torch.nn as nn 
import torch.optim as optim 

c:\Users\woojin\anaconda3\envs\reinforce\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
HIDDEN_SIZE = 128
BATCH_SIZE = 16
PERCENTILE = 70

In [3]:
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )
        
    def forward(self, x):
        return self.net(x)

In [4]:
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])

In [10]:
def iterate_bathces(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs = env.reset()
    sm = nn.Softmax(dim=1)
    while True:
        obs_v = torch.FloatTensor([obs])
        act_probs_v = sm(net(obs_v))
        act_probs = act_probs_v.data.numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, _ = env.step(action)
        episode_reward += reward
        step = EpisodeStep(observation=obs, action=action)
        episode_steps.append(step)
        if is_done:
            e = Episode(reward=episode_reward, steps=episode_steps)
            batch.append(e)
            episode_reward = 0.0
            episode_steps = []
            next_obs = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs

In [21]:
def filter_batch(batch, percentile):
    rewards = list(map(lambda s: s.reward, batch))
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards))
    
    train_obs = []
    train_act = []
    for reward, steps in batch:
        if reward < reward_bound:
            continue
        train_obs.extend(map(lambda step: step.observation, steps))
        train_act.extend(map(lambda step: step.action, steps))
    
    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)
    return train_obs_v, train_act_v, reward_bound, reward_mean

In [22]:
env = gym.make('CartPole-v0')
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n

In [23]:
net = Net(obs_size, HIDDEN_SIZE, n_actions)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.01)
writer = SummaryWriter(comment='-cartpole')

In [24]:
for iter_no, batch in enumerate(iterate_bathces(
    env, net, BATCH_SIZE)):
    obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)
    optimizer.zero_grad()
    action_scores_v = net(obs_v)
    loss_v = objective(action_scores_v, acts_v)
    loss_v.backward()
    optimizer.step()
    print("%d: loss=%.3f, reward_mean=%.1f, rw_bound=%.1f"
          %(iter_no, loss_v.item(), reward_m, reward_b))
    writer.add_scalar("loss", loss_v.item(), iter_no)
    writer.add_scalar("reward_bound", reward_b, iter_no)
    writer.add_scalar("reward_mean", reward_m, iter_no)
    
    if reward_m > 199:
        print("Solved!")
        break
writer.close()

0: loss=0.701, reward_mean=21.1, rw_bound=18.0
1: loss=0.682, reward_mean=19.4, rw_bound=20.0
2: loss=0.681, reward_mean=19.3, rw_bound=23.0
3: loss=0.667, reward_mean=23.1, rw_bound=29.0
4: loss=0.664, reward_mean=35.6, rw_bound=41.5
5: loss=0.656, reward_mean=34.0, rw_bound=35.0
6: loss=0.651, reward_mean=37.2, rw_bound=38.0
7: loss=0.648, reward_mean=39.4, rw_bound=41.5
8: loss=0.632, reward_mean=48.6, rw_bound=57.0
9: loss=0.624, reward_mean=49.8, rw_bound=58.0
10: loss=0.628, reward_mean=45.8, rw_bound=49.5
11: loss=0.629, reward_mean=49.4, rw_bound=56.0
12: loss=0.621, reward_mean=51.8, rw_bound=59.0
13: loss=0.649, reward_mean=53.6, rw_bound=68.0
14: loss=0.632, reward_mean=56.6, rw_bound=65.5
15: loss=0.595, reward_mean=67.9, rw_bound=82.5
16: loss=0.590, reward_mean=68.1, rw_bound=83.5
17: loss=0.590, reward_mean=57.3, rw_bound=63.5
18: loss=0.586, reward_mean=54.4, rw_bound=64.0
19: loss=0.615, reward_mean=53.1, rw_bound=60.5
20: loss=0.599, reward_mean=50.4, rw_bound=55.0
21

In [25]:
# #!/usr/bin/env python3
# import gym
# from collections import namedtuple
# import numpy as np
# from torch.utils.tensorboard import SummaryWriter

# import torch
# import torch.nn as nn
# import torch.optim as optim


# HIDDEN_SIZE = 128
# BATCH_SIZE = 16
# PERCENTILE = 70


# class Net(nn.Module):
#     def __init__(self, obs_size, hidden_size, n_actions):
#         super(Net, self).__init__()
#         self.net = nn.Sequential(
#             nn.Linear(obs_size, hidden_size),
#             nn.ReLU(),
#             nn.Linear(hidden_size, n_actions)
#         )

#     def forward(self, x):
#         return self.net(x)


# Episode = namedtuple('Episode', field_names=['reward', 'steps'])
# EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])


# def iterate_batches(env, net, batch_size):
#     batch = []
#     episode_reward = 0.0
#     episode_steps = []
#     obs = env.reset()
#     sm = nn.Softmax(dim=1)
#     while True:
#         obs_v = torch.FloatTensor([obs])
#         act_probs_v = sm(net(obs_v))
#         act_probs = act_probs_v.data.numpy()[0]
#         action = np.random.choice(len(act_probs), p=act_probs)
#         next_obs, reward, is_done, _ = env.step(action)
#         episode_reward += reward
#         step = EpisodeStep(observation=obs, action=action)
#         episode_steps.append(step)
#         if is_done:
#             e = Episode(reward=episode_reward, steps=episode_steps)
#             batch.append(e)
#             episode_reward = 0.0
#             episode_steps = []
#             next_obs = env.reset()
#             if len(batch) == batch_size:
#                 yield batch
#                 batch = []
#         obs = next_obs


# def filter_batch(batch, percentile):
#     rewards = list(map(lambda s: s.reward, batch))
#     reward_bound = np.percentile(rewards, percentile)
#     reward_mean = float(np.mean(rewards))

#     train_obs = []
#     train_act = []
#     for reward, steps in batch:
#         if reward < reward_bound:
#             continue
#         train_obs.extend(map(lambda step: step.observation, steps))
#         train_act.extend(map(lambda step: step.action, steps))

#     train_obs_v = torch.FloatTensor(train_obs)
#     train_act_v = torch.LongTensor(train_act)
#     return train_obs_v, train_act_v, reward_bound, reward_mean


# if __name__ == "__main__":
#     env = gym.make("CartPole-v0")
#     # env = gym.wrappers.Monitor(env, directory="mon", force=True)
#     obs_size = env.observation_space.shape[0]
#     n_actions = env.action_space.n

#     net = Net(obs_size, HIDDEN_SIZE, n_actions)
#     objective = nn.CrossEntropyLoss()
#     optimizer = optim.Adam(params=net.parameters(), lr=0.01)
#     writer = SummaryWriter(comment="-cartpole")

#     for iter_no, batch in enumerate(iterate_batches(
#             env, net, BATCH_SIZE)):
#         obs_v, acts_v, reward_b, reward_m = \
#             filter_batch(batch, PERCENTILE)
#         optimizer.zero_grad()
#         action_scores_v = net(obs_v)
#         loss_v = objective(action_scores_v, acts_v)
#         loss_v.backward()
#         optimizer.step()
#         print("%d: loss=%.3f, reward_mean=%.1f, rw_bound=%.1f" % (
#             iter_no, loss_v.item(), reward_m, reward_b))
#         writer.add_scalar("loss", loss_v.item(), iter_no)
#         writer.add_scalar("reward_bound", reward_b, iter_no)
#         writer.add_scalar("reward_mean", reward_m, iter_no)
#         if reward_m > 199:
#             print("Solved!")
#             break
#     writer.close()